In [1]:
from pyspark.sql import SparkSession
import pyspark
import findspark

findspark.init()
pyspark.SparkContext.setSystemProperty('spark.executor.memory', '4g')
sc = pyspark.SparkContext(appName="test")
spark_sql = SparkSession.builder.appName("test").getOrCreate()

In [2]:
from bicis.etl.unify_raw_data import UnifyRawData
input_fname = UnifyRawData().output().path.replace('.csv', '_sample.csv')

In [3]:
from pyspark.sql import Row

def _translate_doc(doc):
    res = doc.asDict()
    for date_field in 'rent_date return_date'.split():
        res[date_field + '_date'] = doc[date_field].date()
        res[date_field + '_hour'] = doc[date_field].hour
        res[date_field + '_weekday'] = doc[date_field].isoweekday()
    return Row(**res)

In [4]:
df = (
    spark_sql
    .read
    .load(input_fname, format="csv", sep=",", inferSchema="true", header="true")
    .rdd
    .map(_translate_doc)
    .toDF()
)

